In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import numpy as np

def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()

In [ ]:
model_name="kanhatakeyama/01b_model_30b_token"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto",
    torch_dtype=torch.float16
)


In [ ]:
text_list=[
    "吾輩は猫である｡",
    "最高の気分ですよ!",
    "こんいちは､元気?",
    "こんにちは､元気?",
    "fjalkjfepiwofe",
    "This is the model card of a 🤗 transformers model that has been pushed on the Hub. This model card has been automatically generated.",
]

In [ ]:
for txt in text_list:
    ppl=perplexity(model,tokenizer,txt)
    print(ppl,txt.replace("\n",""))

In [ ]:
from datasets import load_dataset
#mc4の読み込み
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)

dataset=mc4_dataset

In [ ]:
import re
line_list=[]
ppl_list=[]
for i,record in enumerate(dataset):
    txt=record["text"]
    print("-----")

    for line in re.split(r'[。\\n]+', txt):
        if line=="":
            continue
        line=line[:2000]
        line=line.strip()
        ppl=perplexity(model,tokenizer,line)
        print(ppl,line.replace("\n",""))
        line_list.append(line)
        ppl_list.append(ppl)

    if i>15:
        break

In [ ]:
import pandas as pd
df=pd.DataFrame({"text":line_list,"ppl":ppl_list}).sort_values("ppl")
df=df.drop_duplicates("text")
df[-30:]

In [ ]:

df[120:200]